# First Practical Work

## Data Science and Engineering

### DESIGN OF AN IMAGE FILTER FUNCTION, PARALLELIZABLE AND SCALABLE

**Full name:** Mireia Alba Kesti Izquierdo 
    
**NIA**:** 100406960

**Full name:** Aleksandra Jamróz
    
**NIA**:** 100491363

Python multiprocessing documentation https://docs.python.org/3/library/multiprocessing.html

### Setting up the environment

In [ ]:
# import necessary libraries & modules

import numpy as np
import multiprocessing as mp
from PIL import Image               # pillow library - for image manipulation
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cProfile
import ctypes

In [ ]:
# importing our own module (myfunctions)
import myfunctions2 as my2 
import myfunctions as my
import importlib
importlib.reload(my)

# it should not be necessary to import my_functions2, but else it does not work for me
# it complains indicating that it is likely that there is a circular import from partially initialized module "myfunctions"

### Image preparation

In [ ]:
# define the images we want to filter
# if error -> check that they are all in the folder!
chess_file = "chess.jpg"
digits_file = "digits.jpg"
fence_file = "fence.jpg"
hand_x_ray_file = "hand-x-ray.jpg"
jupiter_file = "cloudsonjupi.jpg"

# open the image we have decided to filter
F_IMAGE = Image.open(chess_file)

In [ ]:
# define the colourmap
cmap = None

# get some information about our chosen image
print(F_IMAGE.format) #output: JPEG
print(F_IMAGE.size) #output: (216,233)
print(F_IMAGE.mode) #output: RGB

In [ ]:
# load the image & convert it to a numpy array
image = np.array(F_IMAGE)

In [ ]:
# check that images were converted into numpy arrays correctly
print(type(image)) # --> output should be: <class 'numpy.ndarray'>

In [ ]:
# check if our image is grayscale
# if yes, change the colourmap (i.e. convert the image into a grayscale version of itself)
# if yes, also expand one dimension (if necessary)
if len(image.shape) == 2:
    cmap = "gray"
    image = np.expand_dims(image, axis = 2)

In [ ]:
# create the figure
plt.figure()

# load the image, convert it to a numpy array of pixels & display it
plt.imshow(image, cmap = cmap)

In [ ]:
# get the value of each pixel of the numpy array image
# print(image)

### Filter definition

First, we must define the filters that we are going to apply to our image to filter it. We will define the filters as numpy arrays.
- Filter 1: impulse response filter, i.e. the output image must be equal to the original image.
- Filter 2: edge filter, first order in x axis.
- Filter 3: edge filter, first order in y axis.
- Filter 4: edge filter, second order, bi-directional.
- Filter 5: blur gausian filter.

In [ ]:
# define the 5 available filters as numpy arrays

# Filter 1: impulse response filter
filter1 = np.array([[0,0,0,0,0],
                    [0,0,0,0,0],
                    [0,0,1,0,0],
                    [0,0,0,0,0],
                    [0,0,0,0,0]])

# Filter 2: edge filter, first order in x axis
filter2 = np.array(([0.5, 0 , -0.5]))

# Filter 3: edge filter, first order in y axis
filter3 = np.array([[0.5],[0],[0.5]])

# Filter 4: edge filter, second order, bi-directional
filter4 = np.array([[1,0,-1],
                    [2,0,-2],
                    [1,0,-1]])

# Filter 5: blur gausian filter
filter5 = np.array([[0.00078633,0.00655965,0.01330373,0.00655965,0.00078633],
                    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
                    [0.01330373,0.11098164,0.22508352,0.11098164,0.01330373],
                    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
                    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633]])

### Preparation for applying the filters
Before applying the filters we must define certain variables. We have to define how many processes we wish to start, how much space each filtered image is allowed to occupy, and in which shared spaces our image will be stored after the filter is applied. 

In [ ]:
# define the number of processes that will be running. I will set it to two because my computer has 2 cores
NUMPROCESS = 2

In [ ]:
# calculate the size of the data buffer
# also, allocate in memory to the shared space where our filtered image will be stored 
# i.e. compute the size of the image in bytes and print it
data_buffer_size = image.shape[0] * image.shape[1] * image.shape[2]
print("Size of the image: ", data_buffer_size)

#output: Size of the image:  150984

In [ ]:
# define the multiprocessing vectors (shared spaces)
filtered_image1_VECTOR = mp.Array(ctypes.c_byte, data_buffer_size)
filtered_image2_VECTOR = mp.Array(ctypes.c_byte, data_buffer_size)

In [ ]:
print(filtered_image1_VECTOR)

### Application of one filter
Now, we will apply our first filter to our image. We have chosen to apply Filter 2 on this occasion.

In [ ]:
# apply the filter to our image
my.image_filter(image, filter2, NUMPROCESS, filtered_image1_VECTOR)  

In [ ]:
# convert our filtered image to a numpy array and reshape it 
filtered_image1 = my.tonumpyarray(filtered_image1_VECTOR).reshape(image.shape)

In [ ]:
# create the new figure
plt.figure()

# display the filtered image
plt.imshow(filtered_image1, cmap = cmap)

### Application of two filters
This time we will apply two filters to our image at the same time. We have created separate modules for the filtering functions in order to create separate memory spaces. 

In [ ]:
# apply two filters at the same time by executing two parallel processes
# these processes will split the resources available, half will be used by p1 and half by p2

def filters_execution(image: np.array,  
                      filter_mask1: np.array, 
                      filter_mask2: np.array,  
                      numprocessors: int,
                      filtered_image1: mp.Array,
                      filtered_image2: mp.Array ):
    """
    Function invokes 2 different parallel processes, each executing a filter
    on the same image and saving the result to independent memory spaces. Uses
    previous function for filtering.
    """

    # define the two processes that will be applying the filters
    p1 = mp.Process(target = my.image_filter, args = (image, filter_mask1, numprocessors, filtered_image1))
    p2 = mp.Process(target = my.image_filter, args = (image, filter_mask2, numprocessors, filtered_image2))
    
    # start the processes 
    p1.start() 
    p2.start() 
  
    # wait until the processes have finished, then join them 
    p1.join() 
    p2.join()

    return

In [ ]:
# apply our chosen filters (i.e. execute our main filter function)
filters_execution(image, filter3, filter4, NUMPROCESS, filtered_image1_VECTOR, filtered_image2_VECTOR)

# Visualization
Finally, take the filtered images stored in the multiprocessing.vector variables filtered_image1_VECTOR and filtered_image2_VECTOR, convert them to a numpy array with the same shape as the orginal image, and display the results.

In [ ]:
# convert our filtered images to a numpy array and reshape them 
filtered_image1 = my.tonumpyarray(filtered_image1_VECTOR).reshape(image1.shape)
filtered_image2 = my.tonumpyarray(filtered_image2_VECTOR).reshape(image2.shape)

In [ ]:
# create & display the filtered image 1
plt.figure()
plt.imshow(filtered_image1, cmap = cmap)

In [ ]:
# create & display the filtered image 2
plt.figure()
plt.imshow(filtered_image2, cmap=cmap)

## Conclusion
We have harnessed the power of parallelization to apply filters to our image. When we execute using a single thread, we are (often) not making the best use of our available resources, i.e., we are often not taking full advantage of them, leaving them unused. 
Thus, by using thread pools, we can maximize the number of parallel tasks, increasing the efficiency of our program. 
Furthermore, it is important to include locks in order to handle memory access and avoid races, which we want to avoid. 

In the second task, when we are applying both filters simultaneously, we have to split the resources available, so that these tasks can run concurrently. Here, the limitations of our hardware (physical devices) is magnified, because each process can use only half of the available resources. In my case, as my computer only has 2 cores, the impact is larger. This highlights the importance of matching the software to the available hardware. Even the "best" or most "efficient" code, will not be optimal for any hardware. 